<a href="https://colab.research.google.com/github/BrianAriel/AndroidTP/blob/master/HPC/Cuaderno_2_Miercoles_Grupo1_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1 Introducción**
En este cuaderno realizaremos la implementación tanto secuencial como paralela, mediante CUDA, del algoritmo de multiplicación de matrices. El objetivo es demostrar cómo se utiliza HPC en un contexto de simplificación de complejidad de algoritmos.

Cada desarrollo mostrara los tiempos de ejecución por separado, pero no habrá una sección métricas dado que la explicación a dichos tiempos se extenderá en el informe.

Definimos las matrices A, B y C donde C es la matriz resultado de la siguiente operación: A x B = C.

El ejercicio es parametrizable tanto en el tamaño de las matrices involucradas como de la cantidad de hilos GPU a planificar. Además, se incluye manejo de excepciones en caso de introducir un size_matriz negativo y/o el caso en que se intente correr el ejercicio sin instalar pycuda.

Para terminar con este cuaderno, existen las secciones conclusiones, que brindamos nuestro punto de vista con respecto a eventos o deducciones a partir de los resultados, y la bibliografía donde incluimos aquellos sitios que fueron de referencia y ayuda para el ejercicio.

# **2 Armado del ambiente**

En este apartado nos encargamos de instalar el modulo pycuda, ademas de los imports útiles tanto para el desarrollo GPU como CPU y se setean dos matrices base que serviran para que ambas implementaciones traten los mismos datos

In [1]:
#@title 2.1 Instalacion de librerias
!pip install pycuda

     |████████████████████████████████| 1.7MB 8.7MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 71kB 11.0MB/s 
     |████████████████████████████████| 81kB 11.4MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=627172 sha256=a0fddb9771a3028e515b664be855a340556772ff5aeccc5d440fb7b4b54364ff
  Stored in directory: /root/.cache/pip/wheels/d5/55/64/fd4dddcc5f1c25eebd90b5291c3769101dc978c70165685512
Successfully built pycuda
  Created wheel for pytools: filename=pytools-2021.2.7-py2.py3-none-any.whl size=60644 sha256=d8989170a03d5784488534ddad8376f90ed453053edb206dbd29af11715fab05
  Stored in directory: /root/.cache/pip/wheels/a0/b5/e5/e65d25997fd77729b9aa214645add18688483e48bbcbab6ffc
Successfully built pytools


In [9]:
#@title 2.2 Imports y creacion de matrices
#@markdown ---
size_matriz =  300#@param {type:"number"}
#@markdown ---
from datetime import datetime
import sys
import numpy
try :
  import pycuda.driver as cuda
except ModuleNotFoundError:
  sys.exit("No se pudo importar el paquete de cuda, primero debe ejecutar el comando que lo descarga en la sección de armado del ambiente")
import pycuda.autoinit
from pycuda.compiler import SourceModule

#Creamos las matrices
try:
  if size_matriz > 0:
      matriz_a = numpy.random.randint(1, 3 + 1,(size_matriz, size_matriz)).astype(numpy.int32)
      matriz_b = numpy.random.randint(1, 3 + 1,(size_matriz, size_matriz)).astype(numpy.int32)   
  else:
    raise Exception("Las matrices no pueden tener tamaño menor a 0")
except Exception as e:
  sys.exit(e.args)


# **3 Desarrollo GPU**

En esta implementación se ve el procedimiento estandar para ejecutar código en CUDA donde primero se crean las variables en la CPU, se reserva el tamaño de las mismas en la memoria de GPU y se copian desde el host al dispositivo. Definimos nuestra función kernel y la invocamos, copiando el resultado desde el dispositivo al host una vez que finalice la ejecución de los hilos planificados. Al finalizar mostramos los resultados

En cuanto a la lógica del kernel, la misma es la siguiente:
Cada hilo tendrá su índice el cuál se calcula a partir de su número de hilo más su valor de su respectivo bloque. Solo si se trata de un hilo que no fue planificado de más, es decir, que su número de hilo no es mayor al tamaño de la matriz, se realizará lo siguiente:
Cada hilo toma una fila de A y una columna de B, realizando la suma del producto de cada componente de la fila para producir un elemento de C


In [10]:
#@markdown ---
cantidad_hilos_GPU = 16 #@param {type:"slider", min:1, max:32, step:1}
#@markdown ---

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
# --------------------------------------------

tiempo_ejecucion_concurrente = datetime.now()

#Creamos las matrices
matriz_a_cpu = matriz_a
matriz_b_cpu = matriz_b
matriz_c_cpu = numpy.empty_like(matriz_a_cpu)

#Reservamos la memoria en GPU para las matrices
matriz_a_gpu = cuda.mem_alloc(matriz_a_cpu.nbytes)
matriz_b_gpu = cuda.mem_alloc(matriz_b_cpu.nbytes)
matriz_c_gpu = cuda.mem_alloc(matriz_c_cpu.nbytes)

#Copiamos las matrices de memoria principal a memoria GPU
cuda.memcpy_htod(matriz_a_gpu, matriz_a_cpu)
cuda.memcpy_htod(matriz_b_gpu, matriz_b_cpu)
cuda.memcpy_htod(matriz_c_gpu, matriz_c_cpu)

#Definimos la funcion kernel
module = SourceModule("""
__global__ void mult_matrices(int *a, int *b, int *c, int TAM_MATRIZ)
{
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  int idy = threadIdx.y + blockIdx.y * blockDim.y;

  int Pvalue = 0, Aelement, Belement;

  if(idx < TAM_MATRIZ && idy < TAM_MATRIZ)
  {
    for(int k = 0; k < TAM_MATRIZ; ++k)
    {
      Aelement = a[idy * TAM_MATRIZ + k];
      Belement = b[TAM_MATRIZ * k + idx];
      Pvalue += Aelement * Belement;
    }
    c[idy * TAM_MATRIZ + idx] = Pvalue;
  }
}
""")

#Generamos la funcion kernel
kernel = module.get_function("mult_matrices")

#Planificamos la cantidad de hilos segun los parametros
dim_hilo_x = cantidad_hilos_GPU
#Definimos el tamaño del bloque
dim_bloque_x = numpy.int((cantidad_hilos_GPU + size_matriz - 1) / dim_hilo_x)

#Planificamos la cantidad de hilos segun los parametros
dim_hilo_y = cantidad_hilos_GPU
#Definimos el tamaño del bloque
dim_bloque_y = numpy.int((cantidad_hilos_GPU + size_matriz - 1) / dim_hilo_y)

#Llamamos a la funcion kernel
kernel(matriz_a_gpu, matriz_b_gpu, matriz_c_gpu, numpy.int32(size_matriz),
           block=(dim_hilo_x, dim_hilo_y, 1), grid=(dim_bloque_x, dim_bloque_y,1))

#Esperamos la finalizacion de todos los threads planificados
cuda.Context.synchronize()

#Copiamos los resultados alojados en la memoria GPU a la memoria principal
cuda.memcpy_dtoh(matriz_a_cpu,matriz_a_gpu)
cuda.memcpy_dtoh(matriz_b_cpu,matriz_b_gpu)
cuda.memcpy_dtoh(matriz_c_cpu,matriz_c_gpu)

tiempo_ejecucion_concurrente = datetime.now() - tiempo_ejecucion_concurrente
print("Tiempo de ejecucion GPU:", tiempo_en_ms(tiempo_ejecucion_concurrente), "[ms]")

#Imprimimos los resultados
print ("Matriz A:")
print (matriz_a_cpu)

print ("Matriz B:")
print (matriz_b_cpu)

print ("Matriz C:")
print (matriz_c_cpu)

Tiempo de ejecucion GPU: 4.367 [ms]
Matriz A:
[[1 3 1 ... 2 1 3]
 [3 2 3 ... 2 3 3]
 [3 2 2 ... 2 3 3]
 ...
 [3 1 1 ... 3 2 1]
 [2 2 3 ... 1 1 1]
 [2 1 2 ... 2 3 1]]
Matriz B:
[[3 1 2 ... 1 2 2]
 [1 2 1 ... 1 1 1]
 [2 2 2 ... 3 3 2]
 ...
 [1 3 3 ... 2 3 2]
 [3 2 2 ... 1 2 3]
 [1 3 3 ... 2 3 3]]
Matriz C:
[[1212 1189 1137 ... 1133 1192 1164]
 [1229 1221 1141 ... 1174 1209 1188]
 [1236 1201 1175 ... 1159 1245 1206]
 ...
 [1261 1249 1189 ... 1178 1252 1190]
 [1223 1213 1141 ... 1156 1211 1154]
 [1239 1260 1207 ... 1225 1240 1237]]


# **4 Desarrollo CPU**

Es un desarrollo bastante simple: un triple bucle donde el for mayor itera a traves de las filas de A. El primer for anidado itera sobre las columnas de B y el ultimo se encarga de recorrer las filas de B. Finalmente, el resultado se aloja en C, guardando la sumatoria de los productos de cada elemento

In [12]:
matriz_a_cpu = matriz_a
matriz_b_cpu = matriz_b
matriz_c_cpu = numpy.zeros_like(matriz_a_cpu)

tiempo_ejecucion_secuencial = datetime.now()

#Iterar a traves filas de A
for i in range(len(matriz_a_cpu)):
   #Iterar a traves de columnas de B
   for j in range(len(matriz_a_cpu)):
       # iterate a traves de filas de B
       for k in range(len(matriz_a_cpu)):
           matriz_c_cpu[i][j] += matriz_a_cpu[i][k] * matriz_b_cpu[k][j]

tiempo_ejecucion_secuencial = datetime.now() - tiempo_ejecucion_secuencial

print("Tiempo de ejecucion CPU:", tiempo_en_ms(tiempo_ejecucion_secuencial), "[ms]")

#Imprimimos los resultados
print ("Matriz A:")
print (matriz_a_cpu)

print ("Matriz B:")
print (matriz_b_cpu)

print ("Matriz C:")
print (matriz_c_cpu)

Tiempo de ejecucion CPU: 35906.69 [ms]
Matriz A:
[[1 3 1 ... 2 1 3]
 [3 2 3 ... 2 3 3]
 [3 2 2 ... 2 3 3]
 ...
 [3 1 1 ... 3 2 1]
 [2 2 3 ... 1 1 1]
 [2 1 2 ... 2 3 1]]
Matriz B:
[[3 1 2 ... 1 2 2]
 [1 2 1 ... 1 1 1]
 [2 2 2 ... 3 3 2]
 ...
 [1 3 3 ... 2 3 2]
 [3 2 2 ... 1 2 3]
 [1 3 3 ... 2 3 3]]
Matriz C:
[[1212 1189 1137 ... 1133 1192 1164]
 [1229 1221 1141 ... 1174 1209 1188]
 [1236 1201 1175 ... 1159 1245 1206]
 ...
 [1261 1249 1189 ... 1178 1252 1190]
 [1223 1213 1141 ... 1156 1211 1154]
 [1239 1260 1207 ... 1225 1240 1237]]


# **5 Conclusiones**
Consideramos que la implementación paralela del algoritmo de la multiplicación de matrices es de suma importancia teniendo en cuenta las aplicaciones que el cálculo puede tener: análisis de patrones climáticos, realizar operaciones de álgebra lineal, reconocimiento facial, vehículos autónomos y robotica, análisis de grafos entre muchos otros.

Por otro lado, si bien no vamos a detallar en este cuaderno cuestiones de tiempos de ejecución, queríamos marcar que mientras se desarrollaba el mismo notamos que a partir de matrices de tamaño 300X300 la ejecución secuencial empieza a notar mermas en su finalización llegando a tardar 23 segundos en terminar.

La forma prototipada de implementar CUDA, cuyos pasos estan detallados en la sección 3 Desarrollo GPU, denota que la unica lógica a tener en cuenta es aquella explayada en la funcion kernel, dejando al resto del código como un simple seteo de variables y reservación de memoria.

Para finalizar, animamos a quienes visualicen este cuaderno a probar la ejecución GPU (aunque no la CPU dado los tiempos para finalizar) con tamaños de matrices elevados (con valores de 5000x5000 se vuelve mucho mas interesante) para ver y analizar cuánto se tarda en terminar el algoritmo.

# **6 Bibliografía**

---

[1] W.Valiente. SOA_HPC [Online]. Available: https://github.com/wvaliente/SOA_HPC/blob/main/Trabajos_Destacados/Cuaderno%202020%20Arzola%20Fractal%20OpenCL.ipynb

---

[2] W.Valiente. SOA_HPC [Online]. Available: https://github.com/wvaliente/SOA_HPC/blob/main/Trabajos_DestacadosCuaderno%202020%20Cabre%20Bordes%20de%20imagen%20CUDA.ipynb

---

[3] PyCUDA Mutli GPU multiplication [Online]. Available: https://shephexd.github.io/development/2017/02/19/pycuda.html

---

[4] StackOverflow [Online]. Available: https://stackoverflow.com/questions/21130121/pycuda-precision-of-matrix-multiplication-code

---

[5] StackOverflow [Online]. Available: https://stackoverflow.com/questions/15451958/simple-way-to-create-matrix-of-random-numbers

---

[6] NumPy Documentation [Online]. Available: https://numpy.org/doc/stable/reference/generated/numpy.zeros_like.html

---

[7] NumPy Documentation [Online]. Available: https://numpy.org/doc/1.16/reference/routines.random.html